Install dependencies

In [8]:
!pip -q install google-api-python-client google-auth google-auth-httplib2 google-auth-oauthlib

Authenticate + create Docs API client

In [9]:
from google.colab import auth
auth.authenticate_user()

import google.auth
from googleapiclient.discovery import build

SCOPES = ["https://www.googleapis.com/auth/documents"]
creds, _ = google.auth.default(scopes=SCOPES)

docs_service = build("docs", "v1", credentials=creds)

provided markdown

In [10]:
MARKDOWN = r"""
# Product Team Sync - May 15, 2023
## Attendees
- Sarah Chen (Product Lead)
- Mike Johnson (Engineering)
- Anna Smith (Design)
- David Park (QA)
## Agenda
### 1. Sprint Review
* Completed Features
  * User authentication flow
  * Dashboard redesign
  * Performance optimization
    * Reduced load time by 40%
    * Implemented caching solution
* Pending Items
  * Mobile responsive fixes
  * Beta testing feedback integration
### 2. Current Challenges
* Resource constraints in QA team
* Third-party API integration delays
* User feedback on new UI
  * Navigation confusion
  * Color contrast issues
### 3. Next Sprint Planning
* Priority Features
  * Payment gateway integration
  * User profile enhancement
  * Analytics dashboard
* Technical Debt
  * Code refactoring
  * Documentation updates
## Action Items
- [ ] @sarah: Finalize Q3 roadmap by Friday
- [ ] @mike: Schedule technical review for payment integration
- [ ] @anna: Share updated design system documentation
- [ ] @david: Prepare QA resource allocation proposal
## Next Steps
* Schedule individual team reviews
* Update sprint board
* Share meeting summary with stakeholders
## Notes
* Next sync scheduled for May 22, 2023
* Platform demo for stakeholders on May 25
* Remember to update JIRA tickets
---
Meeting recorded by: Sarah Chen
Duration: 45 minutes
""".strip()

conversion code (parse markdown → create doc → format headings/bullets/checkboxes/mentions/footer)

In [11]:
import re
from dataclasses import dataclass
from typing import List, Tuple, Dict, Any

DOC_URL = "https://docs.google.com/document/d/{}/edit"

MENTION_RE = re.compile(r"@\w[\w.-]*")

@dataclass
class Block:
    kind: str
    text: str
    level: int = 0


def _indent_level_from_spaces(line: str) -> int:
    spaces = len(line) - len(line.lstrip(" "))
    return spaces // 2


def parse_markdown(md: str) -> List[Block]:
    if not md or not md.strip():
        raise ValueError("Markdown input is empty.")

    lines = md.splitlines()
    blocks: List[Block] = []
    in_footer = False
    title_done = False

    for raw in lines:
        line = raw.rstrip("\n")

        if line.strip() == "---":
            in_footer = True
            blocks.append(Block("blank", ""))
            continue

        if not line.strip():
            blocks.append(Block("blank", ""))
            continue

        stripped = line.strip()

        if in_footer:
            blocks.append(Block("footer", stripped))
            continue

        if not title_done and stripped.startswith("# "):
            title_done = True
            full_title = stripped[2:].strip()

            if " - " in full_title:
                main, sub = full_title.split(" - ", 1)
                blocks.append(Block("title", main.strip()))
                blocks.append(Block("subtitle", sub.strip()))
            else:
                blocks.append(Block("title", full_title))
            continue

        if stripped.startswith("## "):
            blocks.append(Block("h2", stripped[3:].strip()))
            continue

        if stripped.startswith("### "):
            blocks.append(Block("h3", stripped[4:].strip()))
            continue

        m_cb = re.match(r"^\s*[-*]\s*\[\s*\]\s+(.*)$", line)
        if m_cb:
            blocks.append(Block("checkbox", m_cb.group(1).strip(), _indent_level_from_spaces(line)))
            continue

        m_b = re.match(r"^\s*[-*]\s+(.*)$", line)
        if m_b:
            blocks.append(Block("bullet", m_b.group(1).strip(), _indent_level_from_spaces(line)))
            continue

        blocks.append(Block("text", stripped))

    if not any(b.kind == "title" for b in blocks):
        raise ValueError("No title found.")

    return blocks


def _rgb(r: float, g: float, b: float) -> Dict[str, Any]:
    return {"color": {"rgbColor": {"red": r, "green": g, "blue": b}}}


def create_google_doc(docs_service, title: str) -> str:
    doc = docs_service.documents().create(body={"title": title}).execute()
    return doc["documentId"]

def build_batch_requests(blocks: List[Block]) -> List[Dict[str, Any]]:
    requests = []
    index = 1

    bullet_ranges = []
    checkbox_ranges = []
    mention_ranges = []

    def safe_range(start, end):
        return {"startIndex": start, "endIndex": end - 1}

    def insert(text, level=0):
        nonlocal index
        payload = ("\t" * level) + text + "\n"
        start = index
        end = index + len(payload)

        requests.append({
            "insertText": {
                "location": {"index": start},
                "text": payload
            }
        })

        index = end
        return start, end

    def heading(start, end, style):
        requests.append({
            "updateParagraphStyle": {
                "range": safe_range(start, end),
                "paragraphStyle": {"namedStyleType": style},
                "fields": "namedStyleType"
            }
        })

    for b in blocks:

        if b.kind == "blank":
            insert("")
            continue

        if b.kind == "title":
            s, e = insert(b.text)
            heading(s, e, "HEADING_1")
            continue

        if b.kind == "subtitle":
            s, e = insert(b.text)
            continue

        if b.kind == "h2":
            s, e = insert(b.text)
            heading(s, e, "HEADING_2")
            continue

        if b.kind == "h3":
            s, e = insert(b.text)
            heading(s, e, "HEADING_3")
            continue

        if b.kind == "checkbox":
            s, e = insert(b.text, b.level)
            checkbox_ranges.append((s, e))
            mention_ranges.append((s, b.text))
            continue

        if b.kind == "bullet":
            s, e = insert(b.text, b.level)
            bullet_ranges.append((s, e))
            mention_ranges.append((s, b.text))
            continue

        if b.kind == "footer":
            insert("")
            s, e = insert(b.text)
            continue

        s, e = insert(b.text)
        mention_ranges.append((s, b.text))

    for s, e in bullet_ranges:
        requests.append({
            "createParagraphBullets": {
                "range": safe_range(s, e),
                "bulletPreset": "BULLET_DISC_CIRCLE_SQUARE"
            }
        })

    for s, e in checkbox_ranges:
        requests.append({
            "createParagraphBullets": {
                "range": safe_range(s, e),
                "bulletPreset": "BULLET_CHECKBOX"
            }
        })

    for start, text in mention_ranges:
        for m in MENTION_RE.finditer(text):
            ms = start + m.start()
            me = start + m.end()

            requests.append({
                "updateTextStyle": {
                    "range": {"startIndex": ms, "endIndex": me},
                    "textStyle": {"bold": True},
                    "fields": "bold"
                }
            })

    return requests


def markdown_to_google_doc(docs_service, md: str) -> str:
    blocks = parse_markdown(md)
    doc_title = next(b.text for b in blocks if b.kind == "title")

    doc_id = create_google_doc(docs_service, doc_title)
    requests = build_batch_requests(blocks)

    docs_service.documents().batchUpdate(
        documentId=doc_id,
        body={"requests": requests}
    ).execute()

    return doc_id


doc_id = markdown_to_google_doc(docs_service, MARKDOWN)
print("Created Google Doc:", DOC_URL.format(doc_id))

Created Google Doc: https://docs.google.com/document/d/1FoyGY4TDws5aFGJ7V_PPabD18XZsdh3q9F4IDaqa8MM/edit
